In [3]:
import requests
import pandas as pd
import sqlite3
import json

### Function to get the url API for Authors and return a the response as json

In [4]:
def get_author_result(author):
    result = requests.get(f'https://openlibrary.org/search/authors.json?q={author}')
    if result.status_code == 200:
        return result.json()
    else:
        raise Exception('Error downloading API response')

### Function to get the url API for Titles and return a the response as json

In [5]:
def get_title_result(title):
    result = requests.get(f'https://openlibrary.org/search.json?title={title}')
    if result.status_code == 200:
        return result.json()
    else:
        raise Exception('Error downloading API response')

### Testing funtion to get author data

In [103]:
r = get_author_result('Nassim Taleb')

In [99]:
key_list = ['key', 'name', 'birth_date', 'top_work', 'work_count']

In [100]:
no_result = get_author_result('Dashiell Hammet')

In [64]:
if no_result['numFound'] == 0:
    print('No result for that author please check the spelling')

No result for that author please check the spelling


In [31]:
# Function to get a tuple to enter in the author DB

def tuple_for_query(data):
    author_id = r['docs'][0]['key']
    full_name = r['docs'][0]['name'].split()
    f_name = full_name[0]
    l_name = full_name[1]
    b_date = r['docs'][0]['birth_date']
    t_work = r['docs'][0]['top_work']
    w_count = r['docs'][0]['work_count']
    

    query_tuple = (author_id, f_name, l_name, b_date, t_work, w_count)
    
    return query_tuple

In [104]:
data = tuple_for_query(r)

In [105]:
data

('OL538956A',
 'Nassim',
 'Nicholas',
 '11 September 1960',
 'The Black Swan: The Impact of the Highly Improbable',
 38)

### Testing get title function

In [93]:
title_r = get_title_result('blank slate')

In [94]:
print(title_r['docs'][0]['key'][7:])
print(title_r['docs'][0]['title'])
print(title_r['docs'][0]['first_publish_year'])
print(title_r['docs'][0]['author_key'][0])
print(title_r['docs'][0]['author_name'][0])

OL477826W
The Blank Slate
2002
OL29930A
Steven Pinker


In [56]:
def tuple_query_title(data):
    book_id = data['docs'][0]['key'][7:]
    title = data['docs'][0]['title']
    f_year_pub = data['docs'][0]['first_publish_year']
    author_key = data['docs'][0]['author_key'][0]
    author_name = data['docs'][0]['author_name'][0].split()
    author_f_name = author_name[0]
    author_l_name = author_name[1]
    
    result = (book_id, title, author_f_name, author_l_name, f_year_pub) 
    return result

In [57]:
def tuple_query_authors_books(data):
    author_key = data['docs'][0]['author_key'][0]
    book_id = data['docs'][0]['key'][7:]
    
    result = (author_key, book_id)
    return result

In [58]:
tuple_query_title(title_r)

('OL27448W', 'The Lord of the Rings', 'J.R.R.', 'Tolkien', 1954)

In [73]:
tuple_query_authors_books(title_r)

('OL26320A', 'OL27448W')

## Author DB CSV and DataFrame will use the key in the json response as a primary key 

### author_id | first_name | last_name |  birth_date | top_work | work_count

## Books DB CSV and DataFrame will use the key in the json responce as a primary key

### book_id | title | author_fname | author_lname | first_publish_year | 

## Bridge table will use Author key and Book key to link book and author

### author_id | book_id

## Create DB and tables for the exercise

In [60]:
conn = sqlite3.connect('author_books.db')
c = conn.cursor()

In [61]:
c.execute("""CREATE TABLE authors (
    author_id text PRIMARY KEY,
    first_name text NOT NULL,
    last_name text NOT NULL,
    birth_date text NOT NULL,
    top_work text NOT NULL,
    work_count integer NOT NULL
    )""")

c.execute("""CREATE TABLE books (
    book_id text PRIMARY KEY,
    title text NOT NULL,
    author_fname text NOT NULL,
    author_lname text NOT NULL,
    first_publish_year text NOT NULL
    )""")

c.execute("""CREATE TABLE authors_books (
    author_id text NOT NULL,
    book_id text NOT NULL,
    PRIMARY KEY (author_id, book_id),
    FOREIGN KEY(author_id) REFERENCES authors(author_id),
    FOREIGN KEY(book_id) REFERENCES books(book_id)
    )""")

In [69]:
def db_table_authors_books(conn):
    conn.execute("""
        SELECT a.author_id , b.book_id , b.title, a.first_name, a.last_name
        FROM authors as a
        JOIN authors_books ON a.author_id = authors_books.author_id
        JOIN books as b ON b.book_id = authors_books.book_id
    """)

    result = conn.fetchall()
    
    return result

In [70]:
db_table_authors_books(c)

[]

In [95]:
df = pd.read_sql_query('SELECT * from authors', conn)

In [96]:
df

,author_id,first_name,last_name,birth_date,top_work,work_count
0,OL28314A,Dashiell,Hammett,27 May 1894,The Maltese Falcon,140
1,OL18319A,Mark,Twain,30 November 1835,Adventures of Huckleberry Finn,2899
2,OL1398754A,Malcolm,Gladwell,"September 3, 1963",Blink: The Power of Thinking Without Thinking,28
3,OL26320A,J.R.R.,Tolkien,3 January 1892,The Hobbit,309
4,OL29930A,Steven,Pinker,1954,"The Language Instinct (""Daily Telegraph"" Talki...",36


In [97]:
# Converting pandas to csv file 
df.to_csv('authors.csv',index=False)